In [ ]:
import numpy as np
import torch.nn as nn 
from torchsummary import summary 
from torch.utils.data import DataLoader, Dataset 
from sklearn.metrics import accuracy_score 
import random 
import torch

### Loading Data

In [ ]:
data = np.load('processed_eeg_data.npz')
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']

In [ ]:
# Change data structure to tensor
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

### Model Testing

In [ ]:
class CNNLSTM1(nn.Module):
# Input size is 8282*23*256 num samples x channels x time steps 
    def __init__(self, output_layers=32, kernel_size=3, dropout_rate=0.5):
        super(CNNLSTM1, self).__init__()
        self.conv1 = nn.Conv1d(23, output_layers, kernel_size, padding=1)
        self.conv2 = nn.Conv1d(output_layers, output_layers*2, kernel_size, padding=1)
        
        self.lstm1 = nn.LSTM(output_layers*2, output_layers*4, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(output_layers*4, 64, num_layers=2, batch_first=True)
        
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.ReLU()(self.conv2(x))
        x = x.permute(0, 2, 1)

        x, _ = self.lstm1(x)
        x = self.dropout(x)
        x, _ = self.lstm2(x)
        x = self.dropout(x)
        x = x[:, -1, :]  # Take the last time step

        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
class CNNLSTM2(nn.Module):
    def __init__(self, conv_channels=32, kernel_size=3, dropout_rate=0.5):
        super(CNNLSTM2, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(23, conv_channels, kernel_size=kernel_size, padding=1),
            nn.BatchNorm1d(conv_channels),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(conv_channels, conv_channels*2, kernel_size=kernel_size, padding=1),
            nn.BatchNorm1d(conv_channels*2),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.lstm = nn.LSTM(input_size=conv_channels*2, hidden_size=128, num_layers=1, batch_first=True, bidirectional=True)

        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(128*2, 64) 
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)  
        x = self.conv2(x) 
        x = x.permute(0, 2, 1) 
        x, _ = self.lstm(x)
        x = self.dropout(x[:, -1, :]) 
        x = nn.ReLU()(self.fc1(x))
        return self.fc2(x)


In [ ]:
search = {
    'output_layers': [16, 32, 64],
    'kernel_size': [3, 5, 7],
    'dropout_rate': [0.2, 0.5, 0.7],
    'learning_rate': [0.01, 0.005, 0.001, 0.0001],
    'batch_size': [16, 32, 64, 128]
}